In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [2]:
reviews_train = []
for line in open('./nuestra-data/entrenamiento.txt', 'r',encoding="utf-8"):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('./nuestra-data/prueba.txt', 'r',encoding="utf-8"):
    
    reviews_test.append(line.strip())

In [3]:
reviews_train[:10]

['_vickydesimone pachiquintans -  Tome un milka oreo estoy asi😎. . jajajaj tkm!',
 'AgoosHughes decimelo a mi, me olvidé de comprarme algo en milka😣',
 'Ark_antos tonse cuando las pillas mis padres suelen pillar hacendado, o cosas así',
 'Benji_Mascolo bimbo felice❤️',
 'byBerted integrales???Pues hay pan integral pan tostado integral pan bimbo integral cereales integrales... Ale una clase rapida e intensiba',
 'camaleon92 Xabi, ambos sabemos que es lo que acabara ocurriendo, que me comeré una tableta de milka JAJAJAJAJA',
 'CandeArbizu McDonalds_Ar me muero ooo!!!!!!! Milka oreo soy tu fan😍😍😍😍',
 'ESAHE me recuerda al pan de pitaya... lo preparan en el norte... me tendre que conformar ahorita con unos de Bimbo :/',
 'FDoficial Pame viste el pan bimbo artesano bueno tambien esta para las hamburguesas... una bomba😁🍔🍔',
 'iComxkon_ yo más milka']

In [4]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [5]:
reviews_train_clean[5]

'camaleon xabi ambos sabemos que es lo que acabara ocurriendo que me comeré una tableta de milka jajajajaja'

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 400 else 0 for i in range(800)]

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
#     Accuracy for C=0.01: 0.87472
#     Accuracy for C=0.05: 0.88368
#     Accuracy for C=0.25: 0.88016
#     Accuracy for C=0.5: 0.87808
#     Accuracy for C=1: 0.87648

Accuracy for C=0.01: 0.845
Accuracy for C=0.05: 0.865
Accuracy for C=0.25: 0.855
Accuracy for C=0.5: 0.86
Accuracy for C=1: 0.855


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
final_model = LogisticRegression(C=0.25)
final_model.fit(X, target)
#print ("Final Accuracy: %s" 
#       % accuracy_score(target, final_model.predict(X_test)))
# Final Accuracy: 0.88128

LogisticRegression(C=0.25, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
#     ('excellent', 0.9288812418118644)
#     ('perfect', 0.7934641227980576)
#     ('great', 0.675040909917553)
#     ('amazing', 0.6160398142631545)
#     ('superb', 0.6063967799425831)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)

('milka', 2.125522966656254)
('heineken', 1.3251142247726788)
('con', 0.7946237444648485)
('una', 0.6146555731668629)
('oreo', 0.5812422301061188)
('todo', 0.5467826471236524)
('bien', 0.5288401549865012)
('buena', 0.5218126324932294)
('bimbo', 0.5055737333048287)
('gracias', 0.4906027880170436)
('adidas', -1.5416504499852022)
('bankia', -1.4290457482682355)
('nike', -1.028831022252962)
('arruinaunacitaconpalabras', -0.8750862006951639)
('bbva', -0.7621233958015582)
('no', -0.6688887771648088)
('santander', -0.6251758383538651)
('banco', -0.593122758350777)
('los', -0.43181312448141324)
('que', -0.39537564276852905)
